In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import interp

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, StratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix, roc_curve, auc

In [ ]:
# all data
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/dataset(445)/Random forest/allData.tsv", sep='\t')
print(f"Shape: {df.shape}")
df.head(1)

Shape: (31620, 10)


,userID,age,gender,movieID,name,year,genre1,genre2,genre3,rating
0,747,1,F,1193,One Flew Over the Cuckoo's Nest,1975,Drama,NaN,NaN,5


In [ ]:
# movie data
movies = pd.read_csv("/content/drive/My Drive/Colab Notebooks/dataset(445)/Random forest/movies.tsv", sep='\t')
print(f"Shape: {df.shape}")
movies.head(1)

Shape: (31620, 10)


,movieID,name,year,genre1,genre2,genre3
0,1,Toy Story,1995,Animation,Children's,Comedy


In [ ]:

df_ratings = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataset(445)/Random forest/ratings.csv')
df_ratings.head(1)

,userID,movieID,rating
0,747,1193,5


In [ ]:
df['rating'].mask(df['rating'] < 4, 0, inplace=True)
df['rating'].mask(df['rating'] > 3, 1, inplace=True)

In [ ]:
def recommendation_without_user_info(df, age, gender, number_recommendations):
    '''
    function returns 5 random movies which have been recommended by a gradient boosting classifier
    without user information
    
    @param df: movie dataset 'allData'
    @param age: user age
    @param gender: user gender
    @param number_recommendations: number of recommendations returned
    '''
    # fit
    # ---------------------------------------------------
    # User information before any movie ratings
    X = df[['age', 'gender', 'year', 'genre1', 'genre2', 'genre3']]
   
    y = df['rating']

    # Preprocessing
    # One hot encoding
    dummyvars = pd.get_dummies(X[['gender', 'genre1', 'genre2', 'genre3']])
    # append the dummy variables to df
    X = pd.concat([X[['age', 'year']], dummyvars], axis = 1)

    print("GradientBoostingClassifier")
    gbclf = GradientBoostingClassifier(n_estimators=100)
    gbclf.fit(X=X, y=y)
    
    # predict
    # ---------------------------------------------------
    # concat user age and gender with movie information, and make predictions
    # e.g. user age 25 and male
    X = df[['age', 'gender', 'year', 'genre1', 'genre2', 'genre3']]
    # set age
    X['age'] = age
    dummyvars = pd.get_dummies(X[['gender', 'genre1', 'genre2', 'genre3']])
    # set gender
    dummyvars['gender_F'] = 0
    dummyvars['gender_M'] = 0
    if gender=='M':
        dummyvars['gender_M'] = 1
    elif gender=='F':
        dummyvars['gender_F'] = 1
    # append the dummy variables to df
    X = pd.concat([X[['age', 'year']], dummyvars], axis = 1)

    # make predictions
    y_pred = gbclf.predict(X=X)
    
    # concat predictions to movie information
    df_pred = pd.concat([df[['movieID', 'name']], pd.DataFrame(y_pred, index=df.index, columns=['pred_rating'])], axis = 1)
    # shuffle 5 random movies with rating 1
    df_pred = df_pred[df_pred.pred_rating==1]
    recommendation = df_pred.drop('pred_rating', axis=1).sample(number_recommendations, random_state=10).set_index('movieID')
    
    return recommendation

# test function
print(recommendation_without_user_info(df=df, age=25, gender='F', number_recommendations=5))

GradientBoostingClassifier
                            name
movieID                         
2078           Jungle Book, The 
3035             Mister Roberts 
2506          Other Sister, The 
3159              Fantasia 2000 
3420     ...And Justice for All 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
